In [1]:
# Standard python packages
import os
import numpy as np
import pandas as pd

# LavAtmos 2: Example 1 
### Running LavAtmos 2 for a Bulk Silicate Earth composition for a single surface temperature and total volatile pressure

In this notebook we will show how straightforward it is to use LavAtmos 2 in order to calculate the partial pressures of the vapor species above a melt of bulk silicate earth (BSE) composition under a volatile atmosphere. Start by ensuring that the standard python packages imported above are all present in your build. Then make sure that the path to the LavAtmos module in the lines below is correct. If you are running the code in the docker provided for Thermoengine on the [ENKI](https://enki-portal.gitlab.io/ThermoEngine/) page, then you should not have to change the paths. 

In [2]:
os.chdir('/home/jovyan/ThermoEngine/LavAtmos')
import lavatmos2

### Setting the melt composition
Next up, we import the bulk silicate earth composition that was also used in the [Visscher and Fegley (2013) paper](https://doi.org/10.1088/2041-8205/767/1/L12). You may also chose to enter your own composition by either change the data file from which we are importing, or by defining a new dictionary type object with the weight percentages for each included oxide. If you do the latter, ensure that the dictionary is in the same format as the one printed below. 

In [3]:
vf13_comps_df = pd.read_csv('/home/jovyan/ThermoEngine/LavAtmos/data/input/vf2013_comps.csv',index_col=0)
vf13_comps = {}
for name in vf13_comps_df.columns:
    vf13_comps[name] = vf13_comps_df[name].to_dict()
print(vf13_comps['BSE'])

{'SiO2': 45.4, 'MgO': 36.76, 'Al2O3': 4.48, 'TiO2': 0.21, 'FeO': 8.1, 'CaO': 3.65, 'Na2O': 0.349, 'K2O': 0.031}


### Setting the volatile atmosphere composition
We now determine the volatile atmosphere composition. In this example we use a hydrogen (H) composition that is roughly similar to what you would expect for an atmosphere with a solar-like composition. Similarly to the melt composition, the volatile compostion is given in dictionary format. Note that here we pass it on in terms of mole fractions (maximum of 1). Feel free to try different compositions yourself!

In [4]:
volatile_comp = {
    'C' : 0.023076923,
    'H' : 7e-09,
    'N' : 0.77692307,
    'S' : 0.0001998,
    'P' : 0.1998002
    }

volatile_comp = {
    'C' : 0,
    'H' : 1,
    'N' : 0,
    'S' : 0,
    'P' : 0
    }

### Setting the surface temperature and the total volatile pressure of the system

Lets try a surface temperature of 3000 K and a total volatile atmosphere pressure of 10 bar.

Note: The total volatile pressure is the total pressure of all the volatile species in the atmosphere, but not of the atmosphere as a whole. The total pressure as a whole is determined by adding the total volatile pressure and the pressure of all the vaporised species to each other. 

In [5]:
T_surf = 2109.11 # Kelvin - if you want to do multiple values, make sure that it is an np.array()
P_vol = 10 # Bar

### Initialising LavAtmos

Next up we initialise a LavAtmos system. This makes sure that the necessary thermochemical data is imported and loaded. 

In [6]:
system = lavatmos2.melt_vapor_system()

### Running LavAtmos

Now we have all we have to do is run the `vaporise` from system by passing on the temperature `T` and the compositions.

In [7]:
partial_pressures = system.vaporise(T_surf, P_vol, vf13_comps['BSE'], volatile_comp)

{'SiO2': 45.4, 'MgO': 36.76, 'Al2O3': 4.48, 'TiO2': 0.21, 'FeO': 8.1, 'CaO': 3.65, 'Na2O': 0.349, 'K2O': 0.031}
Calculated: 100%|██████████| 1/1 [12:49<00:00, 769.67s/it]


In [8]:
print(partial_pressures[['C','H','O','S','P']])

              C         H             O             S             P
0  5.310557e-43  0.010356  7.741466e-09  2.086683e-34  1.259777e-33


In [9]:
# for spec in partial_pressures.columns:
#     print(spec)

In [10]:
print(partial_pressures[['O1Si1','O2Si1','H2O1','C1O2','C1O1']])

      O1Si1         O2Si1      H2O1          C1O2          C1O1
0  0.020501  7.124978e-07  0.081617  3.343788e-34  2.005430e-31


In [11]:
lavatmos_partial_pressures = partial_pressures
all_elements = ['e-', 'Al', 'Ar', 'C', 'Ca', 'Cl', 'Co',\
                 'Cr', 'Cu', 'F', 'Fe', 'Ge', 'H', 'He',\
                 'K', 'Mg', 'Mn', 'N', 'Na', 'Ne', 'Ni',\
                 'O', 'P', 'S', 'Si', 'Ti', 'V', 'Zn']

In [12]:
frac = {}
for el in all_elements:
    frac[el] = 0

P_BOA = lavatmos_partial_pressures.sum(axis=1).iloc[0]
mole_fractions = lavatmos_partial_pressures/P_BOA

for spec in mole_fractions.columns:

    # print(f'\n{spec}')

    for el in all_elements:

        if el in spec:

            index = spec.find(el)

            # Check if there's a coefficient present
            if index + len(el) < len(spec) and spec[index + len(el)].isdigit():
                stoi = float(spec[index + len(el)])

            # Ensure inclusion of ions, end of species atoms, and single atoms
            elif (index + len(el) < len(spec) and spec[index + len(el)] == '_')\
                or index + len(el) == len(spec)\
                or spec[index + len(el)].isupper():
                stoi = 1

            # Ensure exclusion of S in Si species (for example)
            else: 
                stoi = 0

            # print(f'{el}: {stoi}*{mole_fractions[spec].iloc[0]} =',stoi * mole_fractions[spec].iloc[0])
            frac[el] += stoi * mole_fractions[spec].iloc[0]
                    # print(frac[el])

In [13]:
print(f'Total pressure: {P_BOA}')
for el in all_elements:
    print(f'{el:<4}:{frac[el]:4e}')

Total pressure: 10.04122000004064
e-  :2.024387e-07
Al  :1.276757e-08
Ar  :0.000000e+00
C   :2.001077e-32
Ca  :5.443358e-09
Cl  :2.001077e-32
Co  :0.000000e+00
Cr  :0.000000e+00
Cu  :0.000000e+00
F   :2.001077e-32
Fe  :2.797399e-03
Ge  :0.000000e+00
H   :1.982693e+00
He  :2.001077e-32
K   :1.151607e-04
Mg  :5.279526e-04
Mn  :0.000000e+00
N   :2.001077e-32
Na  :2.690379e-03
Ne  :0.000000e+00
Ni  :0.000000e+00
O   :1.021057e-02
P   :2.001077e-32
S   :2.001077e-32
Si  :2.041861e-03
Ti  :1.024002e-09
V   :2.001077e-32
Zn  :0.000000e+00
